In [15]:
import pandas as pd
from pymongo import MongoClient

## Connecting to MongoDB

In [6]:
client = MongoClient('mongodb://localhost:27017/')

### Most filled Academies

In [10]:
# Requires the PyMongo package.
# https://api.mongodb.com/python/current

df = pd.DataFrame(client['parcoursup']['Parcoursup'].aggregate([
    {
        '$project': {
            'name': '$fields.g_ea_lib_vx',
            'academy': '$fields.acad_mies',
            'capacity_establishment': '$fields.capa_fin',
            'number_of_applicant': '$fields.voe_tot',
            'number_of_proposition': '$fields.prop_tot',
            'positive_response_from_candidate': '$fields.acc_tot'
        }
    }, {
        '$group': {
            '_id': '$academy',
            'academy_capacity': {
                '$sum': '$capacity_establishment'
            },
            'academy_filled': {
                '$sum': '$positive_response_from_candidate'
            }
        }
    }, {
        '$addFields': {
            'ratio': {
                '$divide': [
                    '$academy_filled', '$academy_capacity'
                ]
            }
        }
    }, {
        '$sort': {
            'ratio': 1
        }
    }, {
        '$limit': 10
    }
]))
df

,_id,academy_capacity,academy_filled,ratio
0,Etranger,80,34,0.425000
1,Martinique,1296,895,0.690586
2,Polynésie Française,1451,1146,0.789800
3,Lyon,16553,13080,0.790189
4,Guadeloupe,1923,1530,0.795632
5,Guyane,557,465,0.834829
6,Limoges,2621,2252,0.859214
7,Rennes,13153,11499,0.874249
8,Besancon,4337,3794,0.874798
9,Corse,504,443,0.878968


### Applications par academie

In [11]:
df = pd.DataFrame(client['parcoursup']['Parcoursup'].aggregate([
    {
        '$group': {
            '_id': '$fields.g_ea_lib_vx',
            'applicants': {
                '$sum': {
                    '$toInt': '$fields.voe_tot'
                }
            }
        }
    }, {
        '$sort': {
            'applicants': -1
        }
    }
]))
df

,_id,applicants
0,Université de Lille,59962
1,Université de Paris,45712
2,Aix Marseille Université - Site d'Aix-en-Provence,41803
3,Université Paris 13 - Antenne de Bobigny,39921
4,Université Paris Nanterre,37260
...,...,...
1982,Lycée Georges Pompidou,0
1983,Lycée Charles Baudelaire,0
1984,Lycée professionnel Camille Claudel,0
1985,Lycée professionnel Thomas-Jean Main,0


Evolution du taux de remplissage des académies

In [12]:
pd.DataFrame(client['parcoursup']['Parcoursup'].aggregate(
    [
    {
        '$group': {
            '_id': '$fields.acad_mies',
            'candidat_accepted_first': {
                '$avg': '$fields.pct_acc_debutpp'
            },
            'candidat_accepted_second': {
                '$avg': '$fields.pct_acc_datebac'
            },
            'candidat_accepted_third': {
                '$avg': '$fields.pct_acc_finpp'
            }
        }
    }, {
        '$project': {
            'first_round': {
                '$round': [
                    '$candidat_accepted_first', 2
                ]
            },
            'second_round': {
                '$round': [
                    '$candidat_accepted_second', 2
                ]
            },
            'third_round': {
                '$round': [
                    '$candidat_accepted_second', 2
                ]
            }
        }
    }, {
        '$sort': {
            'third_round': 1
        }
    }
]
))

,_id,first_round,second_round,third_round
0,Limoges,21.28,67.32,67.32
1,Paris,25.39,68.59,68.59
2,Créteil,19.92,69.28,69.28
3,Mayotte,35.17,73.89,73.89
4,Montpellier,24.71,73.95,73.95
5,Versailles,21.27,74.13,74.13
6,Lyon,23.71,74.44,74.44
7,Aix-Marseille,25.74,75.02,75.02
8,Nice,23.45,75.08,75.08
9,Normandie,23.85,75.43,75.43


### Flux migratoire des néo-bacheliers

In [14]:
pd.DataFrame(client['parcoursup']['Parcoursup'].aggregate([
    {
        '$group': {
            '_id': '$fields.region_etab_aff',
            'candidats': {
                '$sum': '$fields.acc_tot'
            },
            'stayed': {
                '$avg': '$fields.pct_aca_orig_idf'
            }
        }
    }, {
        '$addFields': {
            'migrated': {
                '$subtract': [
                    100, '$stayed'
                ]
            }
        }
    }, {
        '$sort': {
            'migrated': 1
        }
    }
]))

,_id,candidats,stayed,migrated
0,Mayotte,258,98.275000,1.725000
1,Polynésie française,1146,97.170000,2.830000
2,Guadeloupe,1530,96.311905,3.688095
3,Guyane,465,95.812105,4.187895
4,Corse,443,92.171875,7.828125
5,La Réunion,4467,91.451129,8.548871
6,Martinique,895,90.942308,9.057692
7,Ile-de-France,57779,85.140436,14.859564
8,Normandie,10608,84.706912,15.293088
9,Hauts-de-France,25045,81.291537,18.708463


# Importance de la mention

In [20]:
pd.DataFrame(client['parcoursup']['Parcoursup'].aggregate([
    {
        '$project': {
            'academy': '$fields.acad_mies',
            'formation_selective': {
                '$cond': [
                    {
                        '$regexMatch': {
                            'input': '$fields.select_form',
                            'regex': "formation selective"
                        }
                    }, 1, 0
                ]
            },
            'capacity_establishment': '$fields.capa_fin',
            'total_neo_bacheliers': '$fields.acc_neobac',
            'accepted_without_mention': '$fields.pct_sansmention',
            'accepted_with_ab': '$fields.pct_ab',
            'accepted_with_b': '$fields.pct_b',
            'accepted_with_tb': '$fields.pct_tb'
        }
    }
]))

,_id,academy,formation_selective,capacity_establishment,total_neo_bacheliers,accepted_without_mention,accepted_with_ab,accepted_with_b,accepted_with_tb
0,62b02f870858ff2ddfb74502,Poitiers,1,95,46,6.521739,39.130435,43.478261,10.869565
1,62b02f870858ff2ddfb74503,Poitiers,1,65,29,31.034483,48.275862,20.689655,0.000000
2,62b02f870858ff2ddfb74504,Reims,1,85,57,28.070175,28.070175,33.333333,10.526316
3,62b02f870858ff2ddfb74505,Reims,1,158,85,15.294118,37.647059,40.000000,7.058824
4,62b02f870858ff2ddfb74506,Reims,1,38,22,27.272727,36.363636,31.818182,4.545455
...,...,...,...,...,...,...,...,...,...
4571,62b02f8b0858ff2ddfb756dd,Nancy-Metz,0,85,59,22.033898,32.203390,20.338983,25.423729
4572,62b02f8b0858ff2ddfb756de,Versailles,1,24,22,27.272727,40.909091,27.272727,4.545455
4573,62b02f8b0858ff2ddfb756df,Nice,1,12,11,27.272727,27.272727,36.363636,9.090909
4574,62b02f8b0858ff2ddfb756e0,Nantes,0,36,3,33.333333,66.666667,0.000000,0.000000


### Percentage of students with good mention at baccalaureate accepted in establishments

In [21]:
pd.DataFrame(client['parcoursup']['Parcoursup'].aggregate([
    {
        '$group': {
            '_id': '$fields.g_ea_lib_vx',
            'nb_max_candidat': {
                '$first': '$fields.voe_tot'
            },
            'nb_acc_tb_candidat': {
                '$first': '$fields.acc_tb'
            }
        }
    }, {
        '$sort': {
            'nb_acc_tb_candidat': -1,
            'nb_voe_tot': -1
        }
    }, {
        '$project': {
            'percentage_tb': {
                '$cond': [
                    {
                        '$eq': [
                            '$nb_max_candidat', 0
                        ]
                    }, '0', {
                        '$divide': [
                            {
                                '$multiply': [
                                    '$nb_acc_tb_candidat', 100
                                ]
                            }, '$nb_max_candidat'
                        ]
                    }
                ]
            }
        }
    }
]))

,_id,percentage_tb
0,Sciences Po Paris - Institut d'études politiques,13.192009
1,UTC Université de Technologie de Compiègne,4.160849
2,UTT Université de Technologie de Troyes,4.21012
3,IESEG School of Management - Paris La Défense,3.487179
4,INSA Strasbourg,1.347492
...,...,...
1982,Lycée Marcel Cachin,0.0
1983,Lycée des métiers Condorcet Arcachon,0.0
1984,Lycée Louis Blaringhem,0.0
1985,Université de Bourgogne - site de Nevers,0.0


### List Of Establishment

In [22]:
pd.DataFrame(client['parcoursup']['Parcoursup'].aggregate([
    {
        '$group': {
            '_id': '$fields.acad_mies',
            'establishments': {
                '$addToSet': '$fields.g_ea_lib_vx'
            }
        }
    }, {
        '$addFields': {
            'establishment_count': {
                '$size': '$establishments'
            }
        }
    }, {
        '$sort': {
            'establishment_count': -1
        }
    }
]))

,_id,establishments,establishment_count
0,Versailles,"[Université Paris-Saclay, Campus d'Orsay, I.U....",148
1,Lille,"[Lycée des Flandres, Lycée St-Joseph, Lycée po...",136
2,Créteil,"[Lycée Uruguay France, Lycée Marcel Cachin, Un...",129
3,Nantes,"[ICAM Vendée, Lycée Polyvalent Robert Garnier,...",125
4,Rennes,"[Lycée professionnel Roz Glas, Lycée Pierre Me...",124
5,Paris,"[ESCE, Lycée Estienne Esaig, Lycée Saint Miche...",109
6,Normandie,"[Lycée Malherbe, Lycée Sacre-Coeur, Lycée prof...",108
7,Bordeaux,"[IFSI du CH Agen-Nérac, IFSI CH Intercomm Marm...",105
8,Lyon,"[ESSCA School of Management - Lyon, Lycée Lala...",96
9,Toulouse,"[I.U.T de Toulouse - Antenne de Castres, Unive...",92


### Sum of Applicants by academy

In [23]:
pd.DataFrame(client['parcoursup']['Parcoursup'].aggregate([
    {
        '$group': {
            '_id': '$fields.acad_mies',
            'academy_applicants': {
                '$sum': '$fields.nb_voe_pp_at'
            }
        }
    }, {
        '$sort': {
            'academy_applicants': -1
        }
    }, {
        '$limit': 10
    }
]))

,_id,academy_applicants
0,Paris,114034
1,Versailles,92599
2,Créteil,83109
3,Lille,65523
4,Lyon,63078
5,Bordeaux,53054
6,Toulouse,51452
7,Montpellier,45448
8,Aix-Marseille,40697
9,Rennes,39500


### Orientation between private and public schools

In [24]:
pd.DataFrame(client['parcoursup']['Parcoursup'].aggregate([
    {
        '$group': {
            '_id': '$fields.contrat_etab',
            'total_applicants': {
                '$sum': '$fields.voe_tot'
            },
            'of_which_are_women': {
                '$sum': '$fields.voe_tot_f'
            }
        }
    }, {
        '$addFields': {
            'of_which_are_men': {
                '$subtract': [
                    '$total_applicants', '$of_which_are_women'
                ]
            }
        }
    }
]))

,_id,total_applicants,of_which_are_women,of_which_are_men
0,Public,3329441,1862897,1466544
1,Privé enseignement supérieur,144887,64765,80122
2,Privé hors contrat,11292,8703,2589
3,Privé sous contrat d'association,362729,190607,172122


### Insertion of scholarship students

In [25]:
pd.DataFrame(client['parcoursup']['Parcoursup'].aggregate([
    {
        '$project': {
            'academy': '$fields.acad_mies',
            'nbr_place': '$fields.capa_fin',
            'total_applicants': '$fields.voe_tot',
            'nbr_applicant_boursier': {
                '$add': [
                    '$fields.nb_voe_pp_bg_brs', '$fields.nb_voe_pp_bt_brs', '$fields.nb_voe_pp_bp_brs', '$fields.nb_cla_pp_bg_brs', '$fields.nb_cla_pp_bt_brs', '$fields.nb_cla_pp_bp_brs'
                ]
            },
            'nbr_proposition_boursier': {
                '$add': [
                    '$fields.prop_tot_bg_brs', '$fields.prop_tot_bp_brs', '$fields.prop_tot_bt_brs'
                ]
            }
        }
    }
]))

,_id,academy,nbr_place,total_applicants,nbr_applicant_boursier,nbr_proposition_boursier
0,62b02f870858ff2ddfb74502,Poitiers,95,4291,654,64
1,62b02f870858ff2ddfb74503,Poitiers,65,2884,414,94
2,62b02f870858ff2ddfb74504,Reims,85,2292,664,66
3,62b02f870858ff2ddfb74505,Reims,158,3091,952,92
4,62b02f870858ff2ddfb74506,Reims,38,1640,430,49
...,...,...,...,...,...,...
4571,62b02f8b0858ff2ddfb756dd,Nancy-Metz,85,395,150,56
4572,62b02f8b0858ff2ddfb756de,Versailles,24,473,94,16
4573,62b02f8b0858ff2ddfb756df,Nice,12,199,67,11
4574,62b02f8b0858ff2ddfb756e0,Nantes,36,97,18,11
